In [1]:
import os
import re
import glob
import random
import numpy as np
import scipy
import scipy.io as sio
import scipy.ndimage as ndimage
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as patches
from matplotlib.lines import Line2D
from PIL import Image
from ipywidgets import interact, interactive, fixed, interact_manual
import braingeneers
import braingeneers.data.datasets_electrophysiology as ephys
from braingeneers.analysis.analysis import SpikeData, read_phy_files

In [2]:
def analyze_data(start, stop, dataset_number):
    dataset_path = f"/home/jovyan/work/Human_Hippocampus/data/ephys/2023-04-02-hc328_rec/derived/kilosort2/2023_04_02_hc328_{dataset_number}_curated.zip"
    
    # Map dataset_number to the corresponding light_data file
    light_data_files = {
        0: "20230402T140926-2023_04_02_hc328_0_opto_stim_log.csv",
        1: "20230402T141431-2023_04_02_hc328_1_opto_stim_log.csv",
        2: "20230402T142358-2023_04_02_hc328_2_opto_stim_log.csv",
        3: "20230402T142533-2023_04_02_hc328_3_opto_stim_log.csv",
        4: "20230402T142658-2023_04_02_hc328_4_opto_stim_log.csv",
        5: "20230402T142907-2023_04_02_hc328_5_opto_stim_log.csv",
        6: "20230402T144122-2023_04_02_hc328_6_opto_stim_log.csv",
        7: "20230402T145210-2023_04_02_hc328_7_opto_stim_log.csv"
    }

    # Check if the dataset_number is valid
    if dataset_number not in light_data_files:
        raise ValueError("Invalid dataset_number")

    # Get the corresponding light_data file
    light_data_file = light_data_files[dataset_number]

    # Construct the light_data_path
    light_data_path = f"/home/jovyan/work/Human_Hippocampus/data/opto/hc328_20230402T14/{light_data_file}"
    
    # Read the CSV file into a DataFrame
    light_data = pd.read_csv(light_data_path)

    
    # Read the CSV file into a DataFrame
    light_data = pd.read_csv(light_data_path)

    # Rename the "time (sec)" column
    light_data = light_data.rename(columns={"time (sec)": "time change (sec)"})
    
    # Calculate the time change values by subtracting the first value from each subsequent value
    light_data["time change (sec)"] = light_data["time change (sec)"] - light_data.at[0, "time change (sec)"]

    # Modify the values in the "time change (sec)" column based on the dataset number
    if dataset_number == 5:
        light_data["time change (sec)"] = light_data["time change (sec)"] + 10
    elif dataset_number == 6:
        light_data["time change (sec)"] = light_data["time change (sec)"] + 1

    # Rename the "on_duration (frames)" column
    light_data = light_data.rename(columns={"on_duration (frames)": "on_duration (seconds)"})

    # Divide the values in the "on_duration (seconds)" column by 20000
    light_data["on_duration (seconds)"] = light_data["on_duration (seconds)"] / 20000

    # Rename the "off_duration (frames)" column
    light_data = light_data.rename(columns={"off_duration (frames)": "off_duration (seconds)"})

    # Divide the values in the "off_duration (seconds)" column by 20000
    light_data["off_duration (seconds)"] = light_data["off_duration (seconds)"] / 20000

    # Create a list "light_times" from the values in the first column
    light_times = light_data.iloc[:, 0].tolist()
    
    sd = read_phy_files(dataset_path)
    sd_start = sd.subtime(start*1000, stop*1000)

    
    # Check if start is equal to or at most 10 above any number from light_data
    if any(0 <= start - time <= 10 for time in light_times):
        background_color = (0.6, 0.8, 0.4, 0.5)  # Lighter yellowgreen with an opacity of 0.5
    else:
        background_color = None

    def calculate_mean_firing_rates(spike_data):
        mean_firing_rates = []
        for neuron_spikes in spike_data.train:
            num_spikes = len(neuron_spikes)
            time_duration = spike_data.length / 1000  # Assuming spike times are in milliseconds
            firing_rate = num_spikes / time_duration
            mean_firing_rates.append(firing_rate)

        return np.array(mean_firing_rates)
    
    firing_rates = calculate_mean_firing_rates(sd_start)
    
    def firing_plotter(sd):
        neuron_x = []
        neuron_y = []

        for neuron in sd.neuron_data[0].values():
            neuron_x.append(neuron['position'][0])
            neuron_y.append(neuron['position'][1])

        plt.figure(figsize=(8, 6))
        # Increase the 's' parameter to make the scatter points larger
        # Adjust the 'alpha' parameter to make the scatter points less opaque
        plt.scatter(neuron_x, neuron_y, s=firing_rates*10+firing_rates**2.3, alpha=0.3,c ='r')
        
        if background_color:
            plt.gca().set_facecolor(background_color)
        
        plt.xlabel('um')
        plt.ylabel('um')
        plt.title(f"{start} sec.png")  # Adding the title

        # Set fixed limits for x and y axes
        plt.xlim(600, 1500)
        plt.ylim(0, 2200)
        
        plt.savefig(f"/home/jovyan/work/Human_Hippocampus/saved_plots/firing_animations/dataset_{dataset_number}/{start}_sec.png")
        plt.close()
        
    firing_plotter(sd_start)

    return f"/home/jovyan/work/Human_Hippocampus/saved_plots/firing_animations/dataset_{dataset_number}/{start}_sec.png"

def create_animated_gif(dataset_number):
    # TODO: Make the directory a parameter you can set in the function
    
    # Define the directory path
    directory = '/home/jovyan/work/Human_Hippocampus/saved_plots/firing_animations/'

    # Create a subdirectory to save the figures
    figures_directory = os.path.join(directory, f"dataset_{dataset_number}")
    if not os.path.exists(figures_directory):
        os.makedirs(figures_directory)
    dataset_path = f"/home/jovyan/work/Human_Hippocampus/data/ephys/2023-04-02-hc328_rec/derived/kilosort2/2023_04_02_hc328_{dataset_number}_curated.zip"
    
    sd = read_phy_files(dataset_path)
    length = int(sd.length/1000) 
    
    # Iterate over each half-second of the data
    for half_second in range(length * 2):
        start = half_second / 2
        stop = start + 0.5

        # Call analyze_data function
        image_path = analyze_data(start, stop, dataset_number)
        print(f"Generated image: {image_path}")

    # Directory path where the PNG files are located
    directory = figures_directory

    # Create a list of file names in the directory
#     file_list = sorted([filename for filename in os.listdir(directory) if filename.endswith('.png')], key=lambda x: int(re.search(r'\d+', x).group()))
#     file_list = sorted([filename for filename in os.listdir(directory) if filename.endswith('.png')], key=lambda x: int(x.split("_")[0]))
    file_list = sorted([filename for filename in file_list if re.match(r'\d+(\.\d+)?_sec\.png', filename)], key=lambda x: float(re.search(r'\d+(\.\d+)?', x).group()))

    # Create a list to store the image frames
    frames = []

    # Iterate over each file and add it to the frames list
    for filename in file_list:
        # Create the full file path
        file_path = os.path.join(directory, filename)

        # Open the image file and append it to the frames list
        image = Image.open(file_path)
        frames.append(image)

    # Save the frames as an animated GIF
    save_path = "/home/jovyan/work/Human_Hippocampus/saved_plots/firing_animations/" + f"Dataset_{dataset_number}.gif"
    frames[0].save(save_path, format='GIF', append_images=frames[1:], save_all=True, duration=250, loop=0)


In [7]:
# Example usage
create_animated_gif(dataset_number=6)